<a href="https://colab.research.google.com/github/aounallahsara/ML_models/blob/main/R%C3%A9seau_Bay%C3%A9sien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pgmpy==0.1.19

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pgmpy

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score, BDeuScore
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination

import networkx as nx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/Titanic-Dataset.csv')

In [ ]:
df_raw.head()


In [ ]:
df_raw.shape

In [ ]:
titanic = df_raw.copy()
titanic = titanic.drop(['PassengerId','Ticket', 'Cabin', 'Name'],axis=1)
titanic

In [ ]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())
titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])

In [ ]:
print(titanic.isnull().sum())

Discrétisation

In [ ]:
titanic['Age_dic'] = pd.cut(titanic['Age'], bins=[0,12,60,120], labels=['child','adult','senior'])

In [ ]:
titanic['Fare_dic'] = pd.qcut(titanic['Fare'], q=3, labels=['low','med','high'])

In [ ]:
titanic['family_size'] = titanic['SibSp'] + titanic['Parch']
titanic['family_size_dic'] = pd.cut(titanic['family_size'], bins=[-1,0,2,5,50], labels=['alone','small','medium','large'])

In [ ]:
titanic

In [ ]:
titanic_dic = titanic[['Survived', 'Pclass', 'Sex', 'Embarked', 'Age_dic', 'Fare_dic', 'family_size_dic']].copy()

In [ ]:
titanic_dic

In [ ]:
titanic_dic['Sex'] = titanic_dic['Sex'].map({'male':0, 'female':1})

In [ ]:
col = ['Embarked', 'Age_dic', 'Fare_dic', 'family_size_dic']
titanic_dic = pd.get_dummies(titanic_dic, columns=col, dtype=int)

In [ ]:
titanic_dic

In [ ]:

hc = HillClimbSearch(titanic_dic)

best_bic = hc.estimate(scoring_method=BicScore(titanic_dic))
best_k2 = hc.estimate(scoring_method=K2Score(titanic_dic))
best_bdeu = hc.estimate(scoring_method=BDeuScore(titanic_dic))

print("Edges (BIC):", best_bic)
print("Edges (BIC):", best_bic.edges())
print("Edges (K2):", best_k2)
print("Edges (K2):", best_k2.edges())
print("Edges (BDeu):", best_bdeu)
print("Edges (BDeu):", best_bdeu.edges())

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_model_edges_clean(model, title):
    G = nx.DiGraph()
    G.add_nodes_from(model.nodes())
    G.add_edges_from(model.edges())

    # Positionnement clair et stable
    pos = nx.kamada_kawai_layout(G)

    plt.figure(figsize=(10, 7))

    # Dessin des nœuds
    nx.draw_networkx_nodes(
        G, pos,
        node_size=1800,
        node_color="#a8dadc",   # bleu clair
        edgecolors="black"
    )

    # Dessin des arêtes (flèches)
    nx.draw_networkx_edges(
        G, pos,
        arrowstyle='-|>',
        arrowsize=20,
        node_size=1800,
        width=2,
        edge_color="#1d3557"
    )

    # Labels
    nx.draw_networkx_labels(
        G, pos,
        font_size=12,
        font_weight='bold'
    )

    plt.title(title, fontsize=16)
    plt.axis("off")
    plt.show()


# Appel pour chaque structure
plot_model_edges_clean(best_bic, "Structure apprise (Score BIC)")
plot_model_edges_clean(best_k2, "Structure apprise (Score K2)")
plot_model_edges_clean(best_bdeu, "Structure apprise (Score BDeu)")


In [ ]:
print("Colonnes du dataset (pour evidence):")
print(list(titanic_dic.columns))

# Downgrade numpy to a compatible version
!pip install "numpy<2.0.0" --force-reinstall

# Re-import numpy and other libraries, and pgmpy components, to ensure they use the downgraded numpy
import numpy as np
import pandas as pd

import pgmpy
from pgmpy.models import BayesianNetwork # Changed from BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

# Fonction utilitaire pour apprendre les paramètres et afficher CPDs
def learn_and_inspect(model_structure, data, estimator="MLE", bayes_prior_type='BDeu', ess=5, model_name="model"):
    """
    estimator: "MLE" ou "Bayes"
    bayes_prior_type: prior_type pour BayesianEstimator (ex: 'BDeu' ou 'K2')
    ess: equivalent sample size pour BayesianEstimator
    """
    # s'assurer qu'on travaille sur un objet BayesianNetwork
    if not isinstance(model_structure, BayesianNetwork): # Changed from BayesianModel
        learned_model = BayesianNetwork(model_structure.edges()) # Changed from BayesianModel
    else:
        learned_model = model_structure

    # Parameter learning
    if estimator == "MLE":
        learned_model.fit(data, estimator=MaximumLikelihoodEstimator)
    else:
        learned_model.fit(data, estimator=BayesianEstimator,
                          prior_type=bayes_prior_type,
                          equivalent_sample_size=ess)

    # Vérifier la cohérence (CPDs covers all variables etc.)
    try:
        assert learned_model.check_model()
        print(f"[OK] {model_name} : modèle cohérent (check_model passed).")
    except Exception as e:
        print(f"[WARN] {model_name} : check_model a soulevé : {e}")

    # Afficher CPDs (quelques premiers si trop nombreux)
    cpds = learned_model.get_cpds()
    print(f"\n--- CPDs pour {model_name} (total {len(cpds)}) ---")
    for c in cpds:
        print(c)
        print()

    return learned_model

# Apprentissage des paramètres pour chaque structure
model_bic_mle  = learn_and_inspect(best_bic,  titanic_dic, estimator="MLE", model_name="BIC_MLE")
model_k2_bayes = learn_and_inspect(best_k2,   titanic_dic, estimator="Bayes", bayes_prior_type='BDeu', ess=10, model_name="K2_Bayesian")
model_bdeu_mle  = learn_and_inspect(best_bdeu, titanic_dic, estimator="MLE", model_name="BDeu_MLE")